In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import mysql.connector
import json

In [ ]:
from spotify_api import authentication, get_track_features, get_track, get_artist

In [ ]:
authentication()

In [ ]:
#connect mysql database
conn = mysql.connector.connect(
    host="localhost",
    user="root",
    password="210502",
    database="timeless_song"
)
cursor = conn.cursor()


In [ ]:
cursor.execute("SELECT * FROM top_80s")
top_80s = cursor.fetchall()
top_80s = pd.DataFrame(top_80s, columns=cursor.column_names)
top_80s.drop(columns=['target'], inplace=True)
top_80s['Song'] = top_80s['Song'].str.replace('\"', '')
top_80s['Song'] = top_80s['Song'].str.upper()
top_80s = top_80s.rename(columns={'Song': 'song'})
top_80s.rename(columns={'Artist Names': 'artist'}, inplace=True)
top_80s['artist'] = top_80s['artist'].str.replace('\"', '')
top_80s['artist'] = top_80s['artist'].str.upper()
top_80s['song_id'] = top_80s['uri'].str.split(':').str[-1]
top_80s

In [ ]:
get_track_features('3hVrI7snZMTmHCnbwME47u')

In [ ]:
physical = get_track('3hVrI7snZMTmHCnbwME47u')
physical


In [ ]:
artist = get_artist('4BoRxUdrcgbbq1rxJvvhg9')
artist

In [ ]:
artist_ids = [artist['id'] for artist in physical['artists']]
access_token = "BQBBmD9sst-nIKSE9m_4mexN3PEvq-hceNyCLNOkIV9hYTU9IaUcbXtcZdBSkMA7k-tUydTHo4FsEeimxjEZm2oZ7w6_XkL0SLDcPQV28lcHGQSp5jA"

# Fetch additional information about each artist to get genres
genres = []
for artist_id in artist_ids:
    artist_info = get_artist(artist_id)
    genres.extend(artist_info.get('genres', []))

genres = list(set(genres))
print("Genres:", genres)

In [ ]:
#add genre to each song in top_80s by using song_id to get_track to get artist_id to get artist_info to get genres
top_80s['genres'] = None
for index, row in top_80s.iterrows():
    song_id = row['song_id']
    track = get_track(song_id)
    artist_ids = [artist['id'] for artist in track['artists']]
    genres = []
    for artist_id in artist_ids:
        artist_info = get_artist(artist_id)
        genres.extend(artist_info.get('genres', []))
    top_80s.at[index, 'genres'] = genres
top_80s

In [ ]:
top_80s.to_csv('top_80s.csv', index=False)

In [ ]:
cursor.execute("SELECT * FROM top_90s")
top_90s = cursor.fetchall()
top_90s = pd.DataFrame(top_90s, columns=cursor.column_names)
top_90s.drop(columns=['target'], inplace=True)
top_90s['Song'] = top_90s['Song'].str.replace('\"', '')
top_90s['Song'] = top_90s['Song'].str.upper()
top_90s = top_90s.rename(columns={'Song': 'song'})
top_90s.rename(columns={'Artist Names': 'artist'}, inplace=True)
top_90s['artist'] = top_90s['artist'].str.replace('\"', '')
top_90s['artist'] = top_90s['artist'].str.upper()
top_90s

In [ ]:
cursor.execute("SELECT song, artist, genre FROM song_normalize")
song_normalize = cursor.fetchall()
song_normalize = pd.DataFrame(song_normalize, columns=cursor.column_names)
song_normalize['song'] = song_normalize['song'].str.replace('\"', '')
song_normalize['song'] = song_normalize['song'].str.upper()
song_normalize['artist'] = song_normalize['artist'].str.upper()
song_normalize

In [ ]:
top_80s = pd.merge(top_80s, song_normalize, on=['song', 'artist'], how='left')
top_80s

In [ ]:
top_90s = pd.merge(top_90s, song_normalize, on=['song', 'artist'], how='left')
top_90s

In [ ]:
cursor.execute("SELECT title, artist, year, streams FROM stream_count")
stream_count = cursor.fetchall()
stream_count = pd.DataFrame(stream_count, columns=cursor.column_names)
stream_count.rename(columns={'title': 'song'}, inplace=True)
stream_count['song'] = stream_count['song'].str.replace('\"', '')
stream_count['song'] = song_normalize['song'].str.upper()
stream_count['artist'] = stream_count['artist'].str.upper()
stream_count['year'] = stream_count['date'].str.split('-').str[0].astype(int)
stream_count

In [ ]:
stream_count.isna().sum()

In [ ]:
top_2017_2021 = pd.read_csv('charts.csv')
top_2017_2021.rename(columns={'title': 'song'}, inplace=True)
top_2017_2021['year'] = top_2017_2021['date'].str.split('-').str[0].astype(int)
top_2017_2021 = top_2017_2021[['song', 'artist', 'year', 'streams', 'url']]
# top_2017_2021 = top_2017_2021.fillna(0)
top_2017_2021

In [ ]:
top_2017_2021.isna().sum()

In [ ]:
streaming_2021 = top_2017_2021[top_2017_2021['year'] == 2021]

streaming_2021.loc[:,'song'] = streaming_2021.loc[:,'song'].str.replace('\"', '')
streaming_2021.loc[:,'song'] = streaming_2021.loc[:,'song'].str.upper()

streaming_2021.loc[:,'artist'] = streaming_2021.loc[:,'artist'].str.replace('\"', '')
streaming_2021.loc[:,'artist'] = streaming_2021.loc[:,'artist'].str.upper()
streaming_2021

In [ ]:
sum_streaming_2021 = streaming_2021.groupby(['song', 'artist', 'year']).agg({'streams': 'sum', 'url':'first'}).reset_index()

sum_streaming_2021


In [ ]:
streaming_2020 = top_2017_2021[top_2017_2021['year'] == 2020]

streaming_2020.loc[:,'song'] = streaming_2020.loc[:,'song'].str.replace('\"', '')
streaming_2020.loc[:,'song'] = streaming_2020.loc[:,'song'].str.upper()

streaming_2020.loc[:,'artist'] = streaming_2020.loc[:,'artist'].str.replace('\"', '')
streaming_2020.loc[:,'artist'] = streaming_2020.loc[:,'artist'].str.upper()
streaming_2020

In [ ]:
sum_streaming_2020 = streaming_2020.groupby(['song', 'artist', 'year']).agg({'streams': 'sum', 'url':'first'}).reset_index()

sum_streaming_2020

In [ ]:
streaming_2019 = top_2017_2021[top_2017_2021['year'] == 2019]

streaming_2019.loc[:,'song'] = streaming_2019.loc[:,'song'].str.replace('\"', '')
streaming_2019.loc[:,'song'] = streaming_2019.loc[:,'song'].str.upper()

streaming_2019.loc[:,'artist'] = streaming_2019.loc[:,'artist'].str.replace('\"', '')
streaming_2019.loc[:,'artist'] = streaming_2019.loc[:,'artist'].str.upper()
streaming_2019

In [ ]:
sum_streaming_2019 = streaming_2019.groupby(['song', 'artist', 'year']).agg({'streams': 'sum', 'url':'first'}).reset_index()

sum_streaming_2019

In [ ]:
streaming_2018 = top_2017_2021[top_2017_2021['year'] == 2018]

streaming_2018.loc[:,'song'] = streaming_2018.loc[:,'song'].str.replace('\"', '')
streaming_2018.loc[:,'song'] = streaming_2018.loc[:,'song'].str.upper()

streaming_2018.loc[:,'artist'] = streaming_2018.loc[:,'artist'].str.replace('\"', '')
streaming_2018.loc[:,'artist'] = streaming_2018.loc[:,'artist'].str.upper()
streaming_2018

In [ ]:
sum_streaming_2018 = streaming_2018.groupby(['song', 'artist', 'year']).agg({'streams': 'sum', 'url':'first'}).reset_index()

sum_streaming_2018

In [ ]:
streaming_2017 = top_2017_2021[top_2017_2021['year'] == 2017]

streaming_2017.loc[:,'song'] = streaming_2017.loc[:,'song'].str.replace('\"', '')
streaming_2017.loc[:,'song'] = streaming_2017.loc[:,'song'].str.upper()

streaming_2017.loc[:,'artist'] = streaming_2017.loc[:,'artist'].str.replace('\"', '')
streaming_2017.loc[:,'artist'] = streaming_2017.loc[:,'artist'].str.upper()
streaming_2017

In [ ]:
sum_streaming_2017 = streaming_2017.groupby(['song', 'artist', 'year']).agg({'streams': 'sum', 'url':'first'}).reset_index()

sum_streaming_2017

In [ ]:
merged_df = pd.merge(sum_streaming_2021, top_80s, on=['song', 'artist'], how='inner')
merged_df = merged_df.sort_values(by='Hot100 Rank')
merged_df

In [ ]:
merged_df_2 = pd.merge(sum_streaming_2020, top_80s, on=['song', 'artist'], how='inner')
merged_df_2 = merged_df_2.sort_values(by='Hot100 Rank')
merged_df_2

In [ ]:
merged_df_3 = pd.merge(sum_streaming_2019, top_80s, on=['song', 'artist'], how='inner')
merged_df_3 = merged_df_3.sort_values(by='Hot100 Rank')
merged_df_3

In [ ]:
merged_df_4 = pd.merge(sum_streaming_2018, top_80s, on=['song', 'artist'], how='inner')
merged_df_4 = merged_df_4.sort_values(by='Hot100 Rank')
merged_df_4

In [ ]:
merged_df_5 = pd.merge(sum_streaming_2017, top_80s, on=['song', 'artist'], how='inner')
merged_df_5 = merged_df_4.sort_values(by='Hot100 Rank')
merged_df_5

In [ ]:
final_df = pd.concat([merged_df, merged_df_2, merged_df_3, merged_df_4, merged_df_5], axis=0)
final_df = final_df.sort_values(by='streams', ascending=False)
final_df = final_df.drop_duplicates(subset=['song', 'artist'], keep='first')
final_df = final_df.sort_values(by='Hot100 Rank')
print(len(final_df))
final_df